In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install required packages
print("📦 Installing dependencies...")
!pip install -q transformers datasets accelerate peft tqdm pandas numpy
print("✅ Dependencies installed!")

In [ ]:
# Configuration
import os

# Kaggle paths - UPDATE THESE based on your uploaded dataset names
MODEL_PATH = "/kaggle/input/mitre-fine-tuned-model"  # Your uploaded model dataset
DATA_PATH = "/kaggle/input/mitre-datset"  # Your test data dataset
TEST_FILE = f"{DATA_PATH}/test.jsonl"

# Evaluation settings
EVAL_LIMIT = None  # Set to None for full evaluation, or a number like 100 for quick test
MAX_NEW_TOKENS = 256

print("✅ Configuration loaded")
print(f"   Model: {MODEL_PATH}")
print(f"   Test data: {TEST_FILE}")

# Verify paths exist
if os.path.exists(MODEL_PATH):
    print(f"✅ Model found: {len(os.listdir(MODEL_PATH))} files")
else:
    print(f"❌ Model not found at {MODEL_PATH}")
    print("   Please upload your fine_tuned_model as a Kaggle dataset")

if os.path.exists(TEST_FILE):
    print(f"✅ Test file found")
else:
    print(f"❌ Test file not found at {TEST_FILE}")
    print("   Please add your test dataset to Kaggle")

In [ ]:
# Load the fine-tuned model
print("🔄 Loading fine-tuned model...\n")

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model.eval()  # Set to evaluation mode

print(f"✅ Model loaded from: {MODEL_PATH}")
print(f"📊 GPU Memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"📊 Model device: {model.device}")

In [ ]:
# Load test dataset
print("🔄 Loading test dataset...\n")

from datasets import load_dataset

test_dataset = load_dataset('json', data_files={'test': TEST_FILE})['test']

print(f"✅ Test dataset loaded: {len(test_dataset):,} examples")
print(f"\n📋 Dataset columns: {test_dataset.column_names}")
print(f"\n📋 Sample test entry:")
print(f"   Instruction: {test_dataset[0]['instruction'][:100]}...")
print(f"   Input: {test_dataset[0]['input'][:100]}...")
print(f"   Output: {test_dataset[0]['output'][:100]}...")

In [ ]:
# Define evaluation functions
print("🔄 Defining evaluation functions...\n")

def generate_response(model, tokenizer, instruction, input_text, max_new_tokens=256):
    """Generate a response for a given instruction and input."""
    prompt = f"""{instruction}

### Input:
{input_text}

### Response:
"""
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode only the generated part (remove the prompt)
    generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return generated_text.strip()

def calculate_exact_match(pred, target):
    """Calculate exact match accuracy."""
    return 1.0 if pred.strip().lower() == target.strip().lower() else 0.0

def calculate_partial_match(pred, target):
    """Calculate partial match (keyword overlap)."""
    pred_lower = pred.strip().lower()
    target_lower = target.strip().lower()
    
    # Check if major keywords from target appear in prediction
    target_words = set(target_lower.split())
    pred_words = set(pred_lower.split())
    
    if len(target_words) == 0:
        return 0.0
    
    overlap = len(target_words.intersection(pred_words))
    return overlap / len(target_words)

def calculate_f1_score(pred, target):
    """Calculate F1 score based on word overlap."""
    pred_words = set(pred.strip().lower().split())
    target_words = set(target.strip().lower().split())
    
    if len(pred_words) == 0 or len(target_words) == 0:
        return 0.0
    
    overlap = len(pred_words.intersection(target_words))
    
    precision = overlap / len(pred_words) if len(pred_words) > 0 else 0.0
    recall = overlap / len(target_words) if len(target_words) > 0 else 0.0
    
    if precision + recall == 0:
        return 0.0
    
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

print("✅ Evaluation functions defined")

In [ ]:
# Run evaluation
print("🚀 Running evaluation on test set...\n")

from tqdm import tqdm
import time

# Determine sample size
if EVAL_LIMIT is None:
    eval_samples = test_dataset
    print(f"Evaluating on FULL test set: {len(eval_samples):,} examples")
else:
    eval_samples = test_dataset.select(range(min(EVAL_LIMIT, len(test_dataset))))
    print(f"Evaluating on LIMITED test set: {len(eval_samples):,} examples (out of {len(test_dataset):,})")

print(f"This may take a while...\n")

results = []
exact_matches = 0
partial_match_scores = []
f1_scores = []

start_time = time.time()

for i, example in enumerate(tqdm(eval_samples, desc="Evaluating")):
    # Generate prediction
    prediction = generate_response(
        model,
        tokenizer,
        example['instruction'],
        example['input'],
        max_new_tokens=MAX_NEW_TOKENS
    )
    
    # Calculate metrics
    exact_match = calculate_exact_match(prediction, example['output'])
    partial_match = calculate_partial_match(prediction, example['output'])
    f1 = calculate_f1_score(prediction, example['output'])
    
    exact_matches += exact_match
    partial_match_scores.append(partial_match)
    f1_scores.append(f1)
    
    # Store result
    results.append({
        'index': i,
        'instruction': example['instruction'],
        'input': example['input'],
        'expected': example['output'],
        'predicted': prediction,
        'exact_match': exact_match,
        'partial_match': partial_match,
        'f1_score': f1
    })
    
    # Show first few examples
    if i < 5:
        print(f"\n{'='*80}")
        print(f"Example {i+1}:")
        print(f"Instruction: {example['instruction'][:80]}...")
        print(f"Input: {example['input'][:100]}...")
        print(f"Expected: {example['output'][:100]}...")
        print(f"Predicted: {prediction[:100]}...")
        print(f"Metrics: Exact={exact_match}, Partial={partial_match:.2f}, F1={f1:.2f}")

elapsed = time.time() - start_time

print(f"\n{'='*80}")
print(f"✅ Evaluation completed in {elapsed/60:.2f} minutes ({elapsed/len(eval_samples):.2f} sec/example)")

In [ ]:
# Display main accuracy metrics
import numpy as np

partial_scores_array = np.array(partial_match_scores)
f1_scores_array = np.array(f1_scores)

print("\n" + "="*80)
print("📊 EVALUATION RESULTS")
print("="*80)
print(f"\nTest Set Size: {len(eval_samples):,} examples")
print(f"\n🎯 PRIMARY METRICS:")
print(f"   Exact Match Accuracy: {exact_matches / len(eval_samples) * 100:.2f}%")
print(f"   Partial Match (Recall): {partial_scores_array.mean() * 100:.2f}%")
print(f"   F1 Score: {f1_scores_array.mean() * 100:.2f}%")

print(f"\n📈 DETAILED STATISTICS:")
print(f"\nPartial Match Distribution:")
print(f"   Mean: {partial_scores_array.mean():.3f}")
print(f"   Median: {np.median(partial_scores_array):.3f}")
print(f"   Std Dev: {partial_scores_array.std():.3f}")
print(f"   Min: {partial_scores_array.min():.3f}")
print(f"   Max: {partial_scores_array.max():.3f}")

print(f"\nF1 Score Distribution:")
print(f"   Mean: {f1_scores_array.mean():.3f}")
print(f"   Median: {np.median(f1_scores_array):.3f}")
print(f"   Std Dev: {f1_scores_array.std():.3f}")
print(f"   Min: {f1_scores_array.min():.3f}")
print(f"   Max: {f1_scores_array.max():.3f}")

In [ ]:
# Score distribution analysis
print("\n📊 SCORE DISTRIBUTION ANALYSIS\n")

ranges = [
    (0.0, 0.2, "Very Poor (0-20%)"),
    (0.2, 0.4, "Poor (20-40%)"),
    (0.4, 0.6, "Fair (40-60%)"),
    (0.6, 0.8, "Good (60-80%)"),
    (0.8, 1.0, "Excellent (80-100%)")
]

print("Partial Match Score Distribution:")
for low, high, label in ranges:
    count = sum(1 for score in partial_scores_array if low <= score < high)
    percentage = count / len(partial_scores_array) * 100
    bar = "█" * int(percentage / 2)
    print(f"   {label:25} {count:4d} ({percentage:5.1f}%) {bar}")

print("\nF1 Score Distribution:")
for low, high, label in ranges:
    count = sum(1 for score in f1_scores_array if low <= score < high)
    percentage = count / len(f1_scores_array) * 100
    bar = "█" * int(percentage / 2)
    print(f"   {label:25} {count:4d} ({percentage:5.1f}%) {bar}")

In [ ]:
# Show best predictions
print("\n✅ TOP 5 BEST PREDICTIONS (highest F1 scores):\n")

sorted_results = sorted(results, key=lambda x: x['f1_score'], reverse=True)

for i, result in enumerate(sorted_results[:5]):
    print(f"{i+1}. F1={result['f1_score']:.3f}, Partial={result['partial_match']:.3f}, Exact={result['exact_match']}")
    print(f"   Input: {result['input'][:120]}...")
    print(f"   Expected: {result['expected'][:120]}...")
    print(f"   Predicted: {result['predicted'][:120]}...")
    print()

In [ ]:
# Show worst predictions
print("\n❌ TOP 5 WORST PREDICTIONS (lowest F1 scores):\n")

for i, result in enumerate(sorted_results[-5:]):
    print(f"{i+1}. F1={result['f1_score']:.3f}, Partial={result['partial_match']:.3f}, Exact={result['exact_match']}")
    print(f"   Input: {result['input'][:120]}...")
    print(f"   Expected: {result['expected'][:120]}...")
    print(f"   Predicted: {result['predicted'][:120]}...")
    print()

In [ ]:
# Save detailed results
print("💾 Saving evaluation results...\n")

import json
import pandas as pd

# Save as JSON
results_file = "evaluation_results.json"
with open(results_file, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

# Save as CSV
df = pd.DataFrame(results)
csv_file = "evaluation_results.csv"
df.to_csv(csv_file, index=False, encoding='utf-8')

# Save summary statistics
summary = {
    "test_set_size": len(eval_samples),
    "exact_match_accuracy": float(exact_matches / len(eval_samples)),
    "partial_match_mean": float(partial_scores_array.mean()),
    "partial_match_std": float(partial_scores_array.std()),
    "f1_score_mean": float(f1_scores_array.mean()),
    "f1_score_std": float(f1_scores_array.std()),
    "evaluation_time_minutes": elapsed / 60
}

summary_file = "evaluation_summary.json"
with open(summary_file, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2)

print(f"✅ Results saved:")
print(f"   {results_file} ({os.path.getsize(results_file) / 1024:.1f} KB)")
print(f"   {csv_file} ({os.path.getsize(csv_file) / 1024:.1f} KB)")
print(f"   {summary_file}")
print(f"\n📥 Download these files from the 'Output' section")

In [ ]:
# Create visualization (optional - requires matplotlib)
try:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram of partial match scores
    axes[0].hist(partial_scores_array, bins=20, edgecolor='black', alpha=0.7)
    axes[0].axvline(partial_scores_array.mean(), color='red', linestyle='--', label=f'Mean: {partial_scores_array.mean():.2f}')
    axes[0].set_xlabel('Partial Match Score')
    axes[0].set_ylabel('Frequency')
    axes[0].set_title('Distribution of Partial Match Scores')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Histogram of F1 scores
    axes[1].hist(f1_scores_array, bins=20, edgecolor='black', alpha=0.7, color='green')
    axes[1].axvline(f1_scores_array.mean(), color='red', linestyle='--', label=f'Mean: {f1_scores_array.mean():.2f}')
    axes[1].set_xlabel('F1 Score')
    axes[1].set_ylabel('Frequency')
    axes[1].set_title('Distribution of F1 Scores')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('evaluation_distributions.png', dpi=150, bbox_inches='tight')
    print("✅ Visualization saved: evaluation_distributions.png")
    plt.show()
    
except ImportError:
    print("ℹ️ Matplotlib not available - skipping visualization")

## Summary

The evaluation is complete! Key files generated:
- `evaluation_results.json` - Full detailed results
- `evaluation_results.csv` - Results in CSV format
- `evaluation_summary.json` - Summary statistics
- `evaluation_distributions.png` - Score distribution plots

Download these from the Output section to analyze further.